In [1]:
# This is the model in which we write the algorthim to decide how to allocate portfolio weights 
# This will ultimately output a dataset which all the user needs to do is just use the backtesting functionality
# that we end up writing out in a python notebook to get the plots that we need to analyze what we are working on
# class PortfolioModel:
#     def __init__(self, symbols: dict=Basket().get_portfolio()):
#         self.symbols = symbols
#         self.add = Ticker().recommend_tickers()
    
#     def add_tickers(self, tickers: list):
#         if tickers == self.add:

    
#     def get_tickers(self):
#         pass

In [4]:
import retrieve_tickers as rt
import bt

symbols = list(rt.Basket().get_portfolio().keys())

rt.Ticker().recommend_tickers()

# symbols.extend(rt.Ticker().recommend_tickers())

# df = bt.get(symbols,start="2020-01-01")


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ECC%20%20%20%20%20%20%20%20%20%20%20?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ECC+++++++++++&crumb=IuV1UtA7QtG


['YMM', 'CORT', 'DLO', 'GNTX', 'INCY']

In [42]:
import numpy as np
import yfinance as yf
import pandas as pd

# List of tickers to process
tickers = ["CORT", "CVX", "CDLX", "AAPL"]  # Replace with your list of tickers

# Define the metrics and conditions
metrics = ['currentPrice', 'volume', 'marketCap']  # Replace with your actual metrics
greater = {'currentPrice': 5, 'volume': 200000}
less_than = {'marketCap': 15000000000}

# Retrieve the data
ticker_data = {ticker: dict(filter(lambda item: item[0] in metrics, yf.Ticker(ticker).info.items())) for ticker in tickers}

# Initialize arrays to store the metric values
metric_values = {metric: [] for metric in metrics}

# Populate the arrays with the data
for ticker in tickers:
    for metric in metrics:
        value = ticker_data[ticker].get(metric, np.nan)
        metric_values[metric].append(value)

# Convert lists to numpy arrays
metric_arrays = {metric: np.array(values) for metric, values in metric_values.items()}

# Create boolean arrays for the conditions
check_3 = np.ones(len(tickers), dtype=bool)
check_4 = np.ones(len(tickers), dtype=bool)

for metric, x in greater.items():
    check_3 &= metric_arrays[metric] > x

for metric, x in less_than.items():
    check_4 &= metric_arrays[metric] < x

# Combine the conditions
final_check = check_3 & check_4

# Get the final list of tickers that meet the conditions
new_tickers = np.array(tickers)[final_check].tolist()

print(new_tickers)

['CORT', 'CDLX']


In [ ]:
import numpy as np



In [83]:
import yfinance as yf

yf.Ticker("BRK/B").info

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/BRK/B?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=BRK%2FB&crumb=u8kZW62La4e


YFinanceException: Failed to parse json response from Yahoo Finance: {'code': 'Not Found', 'description': 'HTTP 404 Not Found'}

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_torus(ticker,big,rolling_mean,rolling_std,buy_signal,sell_signal,size):
    # Define parametric equations for a torus
    def torus_parametric(u, v, R, r):
        x = (R + r * np.cos(v)) * np.cos(u)
        y = (R + r * np.cos(v)) * np.sin(u)
        z = r * np.sin(v)
        return x, y, z

    # Parameters for the torus
    R = (rolling_mean[ticker] + (2 * rolling_std[ticker])).mean()  # Major radius
    r = rolling_mean[ticker].mean()  # Minor radius

    # Generate torus coordinates
    u = np.linspace(0,big,size)
    v = np.linspace(0,big,size)
    U, V = np.meshgrid(u, v)
    X, Y, Z = torus_parametric(U, V, R, r)

    # Plot the torus
    fig = plt.figure(figsize=(20, 20))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, alpha=0.2, cmap='viridis')

    # Plot the price data
    ax.scatter(X[buy_signal[ticker]], Y[buy_signal[ticker]], Z[buy_signal[ticker]], color='g', label='Buy Signal')
    ax.scatter(X[sell_signal[ticker]], Y[sell_signal[ticker]], Z[sell_signal[ticker]], color='r', label='Sell Signal')
    ax.tick_params(labelsize=10)
    
    plt.show()

big = 2*np.pi

df_filter = df - df.iloc[0]

norm_price = (df_filter - df_filter.min()) / (df_filter.max() - df_filter.min()) * big

rolling_mean = norm_price.rolling(window=50).mean()
rolling_std = norm_price.rolling(window=50).std()

buy_signal = norm_price < rolling_mean - (2 * rolling_std)
sell_signal = norm_price > rolling_mean + (2 * rolling_std)

plot_torus("cort",big,rolling_mean,rolling_std,buy_signal,sell_signal,len(df))

In [ ]:
def action_tickers(ticker,buy_indicator,sell_indicator):
    tick_buy = pd.DataFrame(buy_indicator[ticker])
    tick_buy["BuySignal"] = "Buy"
    tick_buy = tick_buy.rename(columns={ticker:"BuyAction"})

    tick_sell = pd.DataFrame(sell_indicator[ticker])
    tick_sell["SellSignal"] = "Sell"
    tick_sell = tick_sell.rename(columns={ticker:"SellAction"})

    actions = pd.concat([tick_buy,tick_sell],axis=1)
                     
    actions.loc[(actions["BuyAction"] == False) & (actions["SellAction"] == False), ["BuySignal","SellSignal"]] = "Hold"

    buy = actions.loc[(actions["BuySignal"] == "Buy") & (actions["BuyAction"] == True), "BuySignal"]
    sell = actions.loc[(actions["SellSignal"] == "Sell") & (actions["SellAction"] == True), "SellSignal"]
    hold = actions.loc[(actions["BuySignal"] == "Hold") & (actions["SellSignal"] == "Hold"), "BuySignal"]

    actions = pd.concat([buy,sell,hold],axis=0).reset_index()

    actions.columns = ["Date", ticker + "Signal"]

    actions = actions.sort_values(by="Date",ascending=True).reset_index(drop=True)

    actions = actions.drop(["Date"],axis=1)

    return actions

ticker_action = pd.DataFrame(buy_signal.index)

for tick in symbols:
    ticker_action[tick.lower() + "Signal"] = action_tickers(tick.lower(),buy_signal,sell_signal)

date = ticker_action.pop("Date")
ticker_action = ticker_action.set_index(date)

ticker_action.iloc[51:]

In [ ]:
strat = bt.Strategy('tailAction', [bt.algos.SelectWhere(sell_signal),
                                   bt.algos.WeighEqually(),
                                   bt.algos.Rebalance()])

# now we create the Backtest
t = bt.Backtest(strat, df.iloc[51:,:],integer_positions=False, progress_bar=True)

# and let's run it!
res = bt.run(t)

res.plot()

In [ ]:
strat = bt.Strategy('tailAction', [bt.algos.SelectWhere(buy_signal),
                                   bt.algos.WeighEqually(),
                                   bt.algos.Rebalance()])

# now we create the Backtest
t = bt.Backtest(strat, df.iloc[51:,:],integer_positions=False, progress_bar=True)

# and let's run it!
res = bt.run(t)

res.plot()